# **SIRD Parameters Study**

>In order to control the pandemic spread, every country and the entire world has taken specific measures, from full lockdown to herd immunity. \\
This Notebook is a proposition of analysis of the SIRD model parameters through time, to conclude if whether those taken measures are efficient or not.

## Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize
import plotly.graph_objects as go
import yaml
from datetime import datetime
from sklearn import metrics as mt

## SIRD model

In [7]:
# The SIRD model differential equations.
def SIRD(y, t, N, Beta, Gamma, Mu):
    S, I, R, D = y
    dSdt = -(Beta * I * S)/N
    dIdt = (Beta * I * S)/N  - Gamma * I - Mu * I
    dRdt = Gamma * I
    dDdt = Mu * I
    return dSdt, dIdt, dRdt, dDdt

In [8]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D

In [9]:
#Least Squares Method

def QuadraticError(theta0, Sd, Id, Rd, Dd, y0, t, N):
    """ function to pass to optimize.leastsq
        The routine will square and sum the values returned by 
        this function""" 
    [S,I,R,D] = SIRDsim(y0, t, N, theta0)
    errorS = S - Sd
    errorI = I - Id
    errorR = R - Rd
    errorD = D - Dd
    EQ = np.concatenate([errorI,errorR,errorD])
    return EQ

## Parameters' Estimation Automation

> Brazil is composed of numerous States, and even if there is a National Healthcare System, known as the SUS, each State is governed independently and taken measures are not the same through the wide Brazilian territory. \\
Doing so, this study needs to consider each State independently, to correlate if the real epidemic's evolution is related to the taken measures or not. \\
To do so, automatizing the computational process is the best way to be efficient. 

In [28]:
def SIRD_state_parameters(State_Name, theta0, Tf, Step=1):
  """This function is fitting the SIRD model to a Dataset
     and returns the best parameters through time.
     Args:
           - State_Name: Abbreviated name of a State  | string
           - theta0: Initial parameters               | tuple of float Beta, Gamma, Mu
           - Tf: Final time for parameters' study     | int (days)
           - Step: calculation step                   | int (days)
  """

  # Reading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",") 

  # Dataframe for the specific State
  df_state = df[df.state == State_Name].reset_index()

  # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

  # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  # State population
  with open('/content/drive/MyDrive/ISMIN/Semestre 8/Stage International/Instituto Maua de Tecnologia/Maua Internship/05_Notebooks/DataSets/state_pop.yaml') as f:
    state_pop = yaml.load(f)

  if (State_Name != 'TOTAL'):
    N = state_pop[State_Name]['population']

    #Susceptible population
    alpha = (df_state["recovered"][Tf] + df_state["activeCases"][Tf] + df_state["deaths"][Tf])/N
    N = alpha*N
  else:
    N = (df_state["recovered"][Tf] + df_state["activeCases"][Tf] + df_state["deaths"][Tf])

  # Initial Conditions
  S0, I0, R0, D0 = N-1, 1, 0, 0
  y0 = S0, I0, R0, D0

  # Vectors Initialization
  Beta_table =[]
  Gamma_table = []
  Mu_table = []

  # Initialization of an index vector to personalize the computation
  Index_vector = range(10,Tf,Step)
  # It starts from 10 to have enough samples to define parameters, stops at Tf, 
  # and use Step to change the frequency of the computation (since it takes a long time)

  # Time Loop to find parameters through time
  for Tresearch in Index_vector:
  
    # Considering the research duration for the parameters
    df_state_init = df_state.iloc[:Tresearch]
  
    # Data
    Id = df_state_init["activeCases"]
    Rd = df_state_init["recovered"]
    Dd = df_state_init["deaths"]
    Sd = N - Rd - Id - Dd

    # Time vector
    t = np.linspace(0, len(df_state_init.index.values), len(df_state_init.index.values))

    # Model use to find optimal parameters
    (best_theta, kvg) = optimize.leastsq(QuadraticError, theta0, args=(Sd,Id,Rd,Dd,y0,t,N))
    Beta_table.append(best_theta[0])
    Gamma_table.append(best_theta[1])
    Mu_table.append(best_theta[2])

  timeline = df_state['date'].iloc[:Tf]
  data = {'Date': timeline}
  Parameters_df = pd.DataFrame(data)
  Parameters_df['Beta'] = pd.DataFrame(Beta_table, Index_vector)
  Parameters_df['Gamma'] = pd.DataFrame(Gamma_table, Index_vector)
  Parameters_df['Mu'] = pd.DataFrame(Mu_table, Index_vector)

  return Parameters_df

In [11]:
theta0 = [0.44, 0.15, 0.00292]
SIRD_state_parameters('SP', theta0, 300, 2)

,Date,Beta,Gamma,Mu
0,2020-02-25,NaN,NaN,NaN
1,2020-02-26,NaN,NaN,NaN
2,2020-02-27,NaN,NaN,NaN
3,2020-02-28,NaN,NaN,NaN
4,2020-02-29,NaN,NaN,NaN
...,...,...,...,...
295,2020-12-16,NaN,NaN,NaN
296,2020-12-17,0.165624,0.080436,0.004732
297,2020-12-18,NaN,NaN,NaN
298,2020-12-19,0.164782,0.079698,0.004644


>Since the observation that the computation is taking time (almost 40s if all samples are considered), the implementation of a step has been done. \\
The model can now offer the possibility to treat the data faster. \\
However, as illustrated above, gaining time is loosing accuracy, since the step is bigger. \\
Yet, this lost of accuracy would not be visually perceptible by plotting the results.

## The Transmission Rate: $\color{blue}{\beta}$

### Beta plotting function

In [12]:
def Beta_plotting(Parameters_df):

  timeline = Parameters_df['Date']

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      name="Beta",
      x=timeline,
      y=Parameters_df['Beta'],
      mode='markers',
      line=dict(width=3, dash="dash", color="cornflowerblue")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Days',
      yaxis_title='Beta(t)',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text="Evolution of Beta")
  
  fig.show()

### Visualization

In [13]:
Beta_df = SIRD_state_parameters('SP', theta0, 300, 2)
Beta_plotting(Beta_df)

## The Recovery Rate: $\color{green}{\gamma}$

### Beta plotting function

In [22]:
def Gamma_plotting(Parameters_df):

  timeline = Parameters_df['Date']

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      name="Gamma",
      x=timeline,
      y=Parameters_df['Gamma'],
      mode='markers',
      line=dict(width=3, dash="dash", color="forestgreen")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Days',
      yaxis_title='Beta(t)',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text="Evolution of Beta")
  
  fig.show()

### Visualization

In [23]:
Gamma_df = SIRD_state_parameters('SP', theta0, 300, 2)
Gamma_plotting(Gamma_df)

## The Mortality Rate: $\color{red}{\mu}$

### Beta plotting function

In [20]:
def Mu_plotting(Parameters_df):

  timeline = Parameters_df['Date']

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      name="Mu",
      x=timeline,
      y=Parameters_df['Mu'],
      mode='markers',
      line=dict(width=3, dash="dash", color="goldenrod")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Days',
      yaxis_title='Mu(t)',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text="Evolution of Mu")
  
  fig.show()

### Visualization

In [21]:
Mu_df = SIRD_state_parameters('SP', theta0, 300, 2)
Mu_plotting(Mu_df)

## Combined Graphs

In [24]:
def Parameters_plotting(Parameters_df):
  
  timeline = Parameters_df['Date']

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      name="Beta",
      x=timeline,
      y=Parameters_df['Beta'],
      mode='markers',
      line=dict(width=3, dash="dash", color="cornflowerblue")
      ))
  
  fig.add_trace(go.Scatter(
      name="Gamma",
      x=timeline,
      y=Parameters_df['Gamma'],
      mode='markers',
      line=dict(width=3, dash="dash", color="forestgreen")
      ))

  fig.add_trace(go.Scatter(
      name="Mu",
      x=timeline,
      y=Parameters_df['Mu'],
      mode='markers',
      line=dict(width=3, dash="dash", color="goldenrod")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Days',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text="Evolution of the SIRD Parameters")
  
  fig.show()

## Brazilian States

>The process being implemented, the goal is now to study each of the 26 Brazilian States to determine the decision making influence. 

### Acre

In [30]:
Parameters_plotting(SIRD_state_parameters('AC', theta0, 300, 2))

In [31]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:19:54 using Wesley Cota Database


### Alagoas

In [32]:
Parameters_plotting(SIRD_state_parameters('AL', theta0, 300, 2))

In [33]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:20:09 using Wesley Cota Database


### Amapá

In [34]:
Parameters_plotting(SIRD_state_parameters('AP', theta0, 300, 2))

In [35]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:20:28 using Wesley Cota Database


### Amazonas

In [36]:
Parameters_plotting(SIRD_state_parameters('AM', theta0, 300, 2))

In [37]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:20:51 using Wesley Cota Database


### Bahia

In [38]:
Parameters_plotting(SIRD_state_parameters('BA', theta0, 300, 2))

In [39]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:21:06 using Wesley Cota Database


### Ceará

In [40]:
Parameters_plotting(SIRD_state_parameters('CE', theta0, 300, 2))

In [41]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:21:28 using Wesley Cota Database


### Distrito Federal

In [42]:
Parameters_plotting(SIRD_state_parameters('DF', theta0, 300, 2))

In [43]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:21:44 using Wesley Cota Database


### Espírito Santo

In [44]:
Parameters_plotting(SIRD_state_parameters('ES', theta0, 300, 2))

In [45]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:22:00 using Wesley Cota Database


### Goiás

In [46]:
Parameters_plotting(SIRD_state_parameters('GO', theta0, 300, 2))

In [47]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:22:16 using Wesley Cota Database


### Maranhão

In [48]:
Parameters_plotting(SIRD_state_parameters('MA', theta0, 300, 2))

In [49]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:22:34 using Wesley Cota Database


### Mato Grosso

In [50]:
Parameters_plotting(SIRD_state_parameters('MT', theta0, 300, 2))

In [51]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:22:51 using Wesley Cota Database


### Mato Grosso do Sul

In [52]:
Parameters_plotting(SIRD_state_parameters('MS', theta0, 300, 2))

In [53]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:23:06 using Wesley Cota Database


### Minas Gerais

In [54]:
Parameters_plotting(SIRD_state_parameters('MG', theta0, 300, 2))

In [55]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:23:24 using Wesley Cota Database


### Pará

In [56]:
Parameters_plotting(SIRD_state_parameters('PA', theta0, 300, 2))

In [57]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:23:45 using Wesley Cota Database


### Paraíba

In [58]:
Parameters_plotting(SIRD_state_parameters('PB', theta0, 300, 2))

In [59]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:24:07 using Wesley Cota Database


### Paraná

In [60]:
Parameters_plotting(SIRD_state_parameters('PR', theta0, 300, 2))

In [61]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:24:30 using Wesley Cota Database


### Pernambuco

In [62]:
Parameters_plotting(SIRD_state_parameters('PE', theta0, 300, 2))

In [63]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:24:51 using Wesley Cota Database


### Piauí

In [64]:
Parameters_plotting(SIRD_state_parameters('PI', theta0, 300, 2))

In [65]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:25:06 using Wesley Cota Database


### Rio de Janeiro

In [66]:
Parameters_plotting(SIRD_state_parameters('RJ', theta0, 300, 2))

In [67]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:25:30 using Wesley Cota Database


### Rio Grande do Norte

In [68]:
Parameters_plotting(SIRD_state_parameters('RN', theta0, 300, 2))

In [69]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:25:48 using Wesley Cota Database


### Rio Grande do Sul

In [70]:
Parameters_plotting(SIRD_state_parameters('RS', theta0, 300, 2))

In [71]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:26:05 using Wesley Cota Database


### Rondônia

In [72]:
Parameters_plotting(SIRD_state_parameters('RO', theta0, 300, 2))

In [73]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:26:20 using Wesley Cota Database


### Roraima

In [74]:
Parameters_plotting(SIRD_state_parameters('RR', theta0, 300, 2))

In [75]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:26:38 using Wesley Cota Database


### Santa Catarina

In [76]:
Parameters_plotting(SIRD_state_parameters('SC', theta0, 300, 2))

In [77]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:26:54 using Wesley Cota Database


### São Paulo

In [78]:
Parameters_plotting(SIRD_state_parameters('SP', theta0, 300, 2))

In [79]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:27:14 using Wesley Cota Database


### Sergipe

In [80]:
Parameters_plotting(SIRD_state_parameters('SE', theta0, 300, 2))

In [81]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:27:28 using Wesley Cota Database


### Tocantins

In [82]:
Parameters_plotting(SIRD_state_parameters('TO', theta0, 300, 2))

In [83]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 06/05/2021 at 12:27:43 using Wesley Cota Database


## Conclusion

>> As a conclusion, the model proposed in this Notebook is illustrating the several SIRD model parameters through time, for each Brazilian State. \\
In a purpose of analysis, these results need to be combined to taken individual decisions from each State to conclude on the impact of them onto the propagation.